In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#reading csvs from raw data sets
pop2020_df = pd.read_csv("../RawData/pop2020.csv")
pop2019_df=pd.read_csv("../RawData/pop2019.csv", encoding="ISO-8859-1")
rental_df = pd.read_csv("../RawData/Rental_price.csv")
housing_df = pd.read_csv("../RawData/HomePriceFull.csv")

In [3]:
#cleaning popuation data sets (finding common cities from population data sets)
#selecting wanted data and renaming the columns from 2020 population data

pop2020_df = pop2020_df.iloc[:, 1:4]
pop2020_df=pop2020_df.rename(columns={"Name" : "City",
                                      "2020 Pop":"2020 Population"})
pop2020_df = pop2020_df.replace({"New York City": "New York"})

#selecting wanted data and renaming the columns from 2019 population data
columns = 'NAME', 'STNAME', 'POPESTIMATE2018', 'POPESTIMATE2019'
pop2019_df=pop2019_df.loc[:, columns]
pop2019_df=pop2019_df.rename(columns={"NAME" : "City",
                                      "STNAME" : "State",
                                      "POPESTIMATE2018": "2018 Population",
                                      "POPESTIMATE2019" : "2019 Population"})

#merging population data sets

pop2020_df['City'] = pop2020_df['City'] + ' city'
population_df = pd.merge(pop2019_df, pop2020_df, on=["City", "State"], how='inner')
duplicated = population_df.loc[population_df.duplicated(subset=["City", "State"]), "City"]
pop_df = population_df.drop(duplicated.index)
pop_df.reset_index(drop=True, inplace=True)
pop_df["City"] = pop_df["City"].str.replace("city", "")

#abbrev. state names
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

pop_df['State'] = pop_df['State'].map(us_state_abbrev).fillna(pop_df['State'])

#exporting data as csv file 
pop_df.to_csv("cleaned_population.csv")
pop_df

City State  2018 Population  2019 Population 2020 Population
0    Birmingham     AL           210487           209403         208,940
1    Huntsville     AL           198125           200574         202,910
2        Mobile     AL           189809           188720         186,804
3    Montgomery     AL           198899           198525         197,282
4      Chandler     AZ           257186           261165         268,675
..           ...   ...              ...              ...             ...
177     Spokane     WA           219332           222081         223,266
178      Tacoma     WA           216111           217827         222,603
179   Vancouver     WA           182887           184463         186,516
180     Madison     WI           257854           259680         264,030
181   Milwaukee     WI           591375           590157         585,589

[182 rows x 5 columns]

In [5]:
#renaming column name and selecting wanted data from rental.csv
rental_df = rental_df.rename(columns={"RegionName":"City"})
rental_df.drop(rental_df.iloc[:, 2:50], inplace = True, axis = 1) 
del rental_df ["RegionID"]

#spliting state name form city and adding in a new column
rental_df["State"] = rental_df["City"].str.split(',').str[1]
rental_df['City'] = rental_df['City'].str.split(',').str[0]

#rearrnaging the order of the colums 
columns = ['City', 'State','2018-01','2018-02', '2018-03', '2018-04', '2018-05', '2018-06','2018-07', 
           '2018-08','2018-09', '2018-10', '2018-11', '2018-12','2019-01', '2019-02', '2019-03', 
           '2019-04', '2019-05', '2019-06','2019-07', '2019-08', '2019-09', '2019-10', '2019-11', 
           '2019-12','2020-01', '2020-02', '2020-03', '2020-04','2020-05', '2020-06','2020-07', 
           '2020-08', '2020-09']

rental_df=rental_df.loc[:, columns]

#renaming data
rental_df = rental_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
rental_df = pd.merge(pop_df["City"], rental_df, on="City", how="outer")
new_rental_df=rental_df.dropna(how="any")
new_rental_df.reset_index(drop=True, inplace=True)
new_rental_df.to_csv("cleaned_rental.csv")
new_rental_df

City State  2018-01  2018-02  2018-03  2018-04  2018-05  \
0          New York    NY   2598.0   2601.0   2605.0   2608.0   2612.0   
1       Los Angeles    CA   2380.0   2388.0   2397.0   2405.0   2414.0   
2           Chicago    IL   1705.0   1708.0   1710.0   1713.0   1716.0   
3            Dallas    TX   1470.0   1474.0   1478.0   1481.0   1485.0   
4      Philadelphia    PA   1505.0   1509.0   1512.0   1516.0   1521.0   
..              ...   ...      ...      ...      ...      ...      ...   
100  Port St. Lucie    FL   1486.0   1495.0   1503.0   1512.0   1520.0   
101    Fort Collins    CO   1541.0   1545.0   1550.0   1555.0   1559.0   
102         Boulder    CO   1884.0   1890.0   1895.0   1901.0   1906.0   
103         Greeley    CO   1568.0   1574.0   1580.0   1586.0   1592.0   
104     Gainesville    GA   1240.0   1245.0   1249.0   1253.0   1260.0   

     2018-06  2018-07  2018-08  ...  2019-12  2020-01  2020-02  2020-03  \
0     2617.0   2621.0   2626.0  ...   2698.0   2694.0   2686.0   2678.0   
1     2423.0   2432.0   2440.0  ...   2542.0   2544.0   2545.0   2545.0   
2     1719.0   1722.0   1725.0  ...   1778.0   1778.0   1777.0   1776.0   
3     1489.0   1493.0   1498.0  ...   1560.0   1562.0   1563.0   1564.0   
4     1525.0   1530.0   1534.0  ...   1598.0   1600.0   1602.0   1604.0   
..       ...      ...      ...  ...      ...      ...      ...      ...   
100   1529.0   1538.0   1545.0  ...   1640.0   1648.0   1656.0   1664.0   
101   1564.0   1569.0   1574.0  ...   1636.0   1638.0   1641.0   1644.0   
102   1911.0   1916.0   1921.0  ...   1991.0   1992.0   1993.0   1995.0   
103   1598.0   1604.0   1611.0  ...   1680.0   1682.0   1685.0   1688.0   
104   1268.0   1275.0   1284.0  ...   1395.0   1399.0   1402.0   1406.0   

     2020-04  2020-05  2020-06  2020-07  2020-08  2020-09  
0     2671.0   2662.0   2653.0   2644.0   2635.0   2625.0  
1     2546.0   2546.0   2546.0   2546.0   2545.0   2545.0  
2     1776.0   1775.0   1774.0   1772.0   1771.0   1770.0  
3     1566.0   1567.0   1568.0   1570.0   1571.0   1572.0  
4     1606.0   1608.0   1610.0   1612.0   1614.0   1615.0  
..       ...      ...      ...      ...      ...      ...  
100   1671.0   1679.0   1687.0   1695.0   1702.0   1710.0  
101   1646.0   1649.0   1651.0   1653.0   1656.0   1658.0  
102   1996.0   1997.0   1998.0   1998.0   1999.0   2000.0  
103   1690.0   1693.0   1695.0   1698.0   1700.0   1703.0  
104   1410.0   1413.0   1417.0   1420.0   1423.0   1427.0  

[105 rows x 35 columns]

In [6]:
#2018 House price data (renaming the column and rearrange the order)
housing_df = housing_df.rename(columns={"RegionName":"City"})
columns_pull = ['City', 'State', '1/31/18', '2/28/18', '3/31/18', '4/30/18', '5/31/18',
       '6/30/18', '7/31/18', '8/31/18', '9/30/18', '10/31/18', '11/30/18', '12/31/18']
housing_2018 = housing_df[columns_pull]

#exporting data as csv file 
housing_2018.to_csv("cleaned_housing2018.csv")
housing_2018

City State   1/31/18   2/28/18   3/31/18   4/30/18  \
0              New York    NY  598182.0  602011.0  606129.0  610045.0   
1           Los Angeles    CA  670959.0  676364.0  682403.0  687595.0   
2               Houston    TX  173623.0  174120.0  174701.0  175669.0   
3               Chicago    IL  261760.0  263098.0  263089.0  263173.0   
4           San Antonio    TX  175263.0  175192.0  176151.0  177407.0   
...                 ...   ...       ...       ...       ...       ...   
26374          Blockton    IA   28743.0   28551.0   28534.0   28597.0   
26375         Smethport    PA   69372.0   69610.0   69765.0   70026.0   
26376          Winfield    TX   66039.0   66568.0   66777.0   66726.0   
26377  Mount Charleston    NV  333152.0  336097.0  339783.0  341758.0   
26378         Clearview    WV  137429.0  137665.0  138172.0  139166.0   

        5/31/18   6/30/18   7/31/18  8/31/18  9/30/18  10/31/18  11/30/18  \
0      613190.0  616052.0  618337.0   620670   623400    625542    627733   
1      691332.0  694791.0  697835.0   699192   699614    700492    701171   
2      176746.0  177725.0  178558.0   179543   180496    181183    181809   
3      263829.0  265011.0  265445.0   265726   266101    266700    267288   
4      178760.0  179751.0  180688.0   181494   181682    182178    182403   
...         ...       ...       ...      ...      ...       ...       ...   
26374   28879.0   28951.0   29203.0    29040    28753     28088     27726   
26375   70508.0   71168.0   71698.0    72193    72450     72849     73198   
26376   66506.0   66956.0   66716.0    66962    66621     66769     66405   
26377  344323.0  347419.0  352425.0   356781   361223    365172    369765   
26378  139814.0  140257.0  140729.0   141862   142345    141508    140654   

       12/31/18  
0        629057  
1        700421  
2        182548  
3        267608  
4        183796  
...         ...  
26374     27671  
26375     73584  
26376     66197  
26377    373397  
26378    140123  

[26379 rows x 14 columns]

In [7]:
#rearranging columns of 2019 House price data
columns_pull = ['City', 'State', '1/31/19', '2/28/19', '3/31/19', '4/30/19', '5/31/19',
       '6/30/19', '7/31/19', '8/31/19', '9/30/19', '10/31/19', '11/30/19', '12/31/19']
housing_2019 = housing_df[columns_pull]

#exporting data as csv file 
housing_2019.to_csv("cleaned_housing2019.csv")
housing_2019


City State  1/31/19  2/28/19  3/31/19  4/30/19  5/31/19  \
0              New York    NY   629748   629807   630547   630499   631055   
1           Los Angeles    CA   699716   696405   693381   692770   697440   
2               Houston    TX   183815   184795   185234   185159   185422   
3               Chicago    IL   267126   266817   266950   267709   268376   
4           San Antonio    TX   184815   186346   186831   187843   188627   
...                 ...   ...      ...      ...      ...      ...      ...   
26374          Blockton    IA    27897    27977    28075    28094    28283   
26375         Smethport    PA    73843    74132    74418    74840    75165   
26376          Winfield    TX    66006    66389    66912    67627    68413   
26377  Mount Charleston    NV   376029   375455   374664   375051   378511   
26378         Clearview    WV   140543   141096   141851   142621   143201   

       6/30/19  7/31/19  8/31/19  9/30/19  10/31/19  11/30/19  12/31/19  
0       630202   630091   628669   627101    625656    624568    624329  
1       703920   705113   704907   704465    707935    710133    714109  
2       186223   187154   187964   188095    188369    188440    188921  
3       268754   268861   268746   267868    267303    266611    266519  
4       189666   190293   190770   191362    191720    192199    192325  
...        ...      ...      ...      ...       ...       ...       ...  
26374    28498    28407    28397    28669     29453     30226     30665  
26375    75752    76104    76470    76566     76692     76726     76745  
26376    68600    68858    68811    69470     70051     70551     70929  
26377   382142   385076   383238   380270    377735    378425    380050  
26378   143824   144155   144418   144568    145286    145658    146075  

[26379 rows x 14 columns]

In [8]:
#rearranging columns 2020 House price data
columns_pull = ['City', 'State','1/31/20', '2/29/20', '3/31/20', '4/30/20', '5/31/20',
       '6/30/20', '7/31/20', '8/31/20', '9/30/20']
housing_2020 = housing_df[columns_pull]

#exporting data as csv file 
housing_2020.to_csv("cleaned_housing2020.csv")
housing_2020

City State  1/31/20  2/29/20  3/31/20  4/30/20  5/31/20  \
0              New York    NY   624680   624947   624563   625025   625187   
1           Los Angeles    CA   718163   724618   731034   736655   738996   
2               Houston    TX   189055   189529   190455   191618   192630   
3               Chicago    IL   266784   267377   268315   269091   269955   
4           San Antonio    TX   192858   193046   193730   194053   194575   
...                 ...   ...      ...      ...      ...      ...      ...   
26374          Blockton    IA    30918    31330    31698    32124    32390   
26375         Smethport    PA    76853    76949    77006    76844    76595   
26376          Winfield    TX    71164    71252    71322    71433    71565   
26377  Mount Charleston    NV   381352   383120   384323   385889   384576   
26378         Clearview    WV   146435   146917   147155   147246   147623   

       6/30/20  7/31/20  8/31/20  9/30/20  
0       626609   627671   629936   632271  
1       740780   747509   757047   768046  
2       193385   194246   195245   196902  
3       271134   273126   275564   278816  
4       195057   195872   196886   198162  
...        ...      ...      ...      ...  
26374    32730    33279    33914    34391  
26375    76003    75538    74984    74649  
26376    71937    72561    73178    73572  
26377   382582   380354   382602   385977  
26378   148060   148680   149077   149831  

[26379 rows x 11 columns]